# HW1: Frame-Level Speech Recognition

## Overview

This project involves building a model to perform frame-level speech recognition using MFCC data consisting of 28 features per time step. The objective is to recognize the phoneme that occurs in each frame.

## Experiments and Results

- **Model Architecture**:
  The model that gave the best result uses a fully connected architecture with batch normalization and dropout layers to improve generalization. Here are the details:

  ```plaintext
  Network(
    (model): Sequential(
      (0): Linear(in_features=1148, out_features=3072, bias=True)
      (1): BatchNorm1d(3072)
      (2): ReLU()
      (3): Dropout(p=0.1)
      (4): Linear(in_features=3072, out_features=2048, bias=True)
      (5): BatchNorm1d(2048)
      (6): ReLU()
      (7): Dropout(p=0.1)
      (8): Linear(in_features=2048, out_features=1024, bias=True)
      (9): BatchNorm1d(1024)
      (10): ReLU()
      (11): Dropout(p=0.1)
      (12): Linear(in_features=1024, out_features=512, bias=True)
      (13): BatchNorm1d(512)
      (14): ReLU()
      (15): Dropout(p=0.1)
      (16): Linear(in_features=512, out_features=256, bias=True)
      (17): BatchNorm1d(256)
      (18): ReLU()
      (19): Dropout(p=0.1)
      (20): Linear(in_features=256, out_features=128, bias=True)
      (21): BatchNorm1d(128)
      (22): ReLU()
      (23): Linear(in_features=128, out_features=42, bias=True)
    )
  )
  ```

  This architecture processes an input size of 1148, which corresponds to the context window of frames concatenated together.

- **Best Hyperparameters**:
  The best accuracy of 86% was achieved using the following configuration:

  - **Epochs**: 20
  - **Batch size**: 1024
  - **Learning rate**: 0.0015
  - **Context window**: 20
  - **Dropout rate**: 0.1

- **Training**:
  The model was trained for 20 epochs using a batch size of 1024. A learning rate scheduler was used to dynamically adjust the learning rate during training, starting at 0.0015.

  ```python
  criterion = torch.nn.CrossEntropyLoss()
  
  optimizer = torch.optim.AdamW(model.parameters(), lr=config['init_lr'], weight_decay=1e-4)
  
  scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=3, T_mult=1, eta_min=1e-6)
  ```  

In this homework, you will be working with MFCC data consisting of 28 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [4]:
!pip install torchsummaryX==1.1.0 wandb --quiet

In [5]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [3]:
''' If you are using colab, you can import google drive to save model checkpoints in a folder
    If you want to use it, uncomment the two lines below
'''
# from google.colab import drive
# drive.mount('/content/drive')

' If you are using colab, you can import google drive to save model checkpoints in a folder\n    If you want to use it, uncomment the two lines below\n'

In [6]:
### PHONEME LIST
PHONEMES = [
    '[SIL]', 'AA', 'AE', 'AH', 'AO', 'AW', 'AY',
    'B', 'CH', 'D', 'DH', 'EH', 'ER', 'EY',
    'F', 'G', 'HH', 'IH', 'IY', 'JH', 'K',
    'L', 'M', 'N', 'NG', 'OW', 'OY', 'P',
    'R', 'S', 'SH', 'T', 'TH', 'UH', 'UW',
    'V', 'W', 'Y', 'Z', 'ZH', '[SOS]', '[EOS]']

# Kaggle

In [ ]:
!rm -rf /root/.kaggle

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"seitomoyi","key":"5165f2111fd03e4eb81eea7405f4b319"}')
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!cat /root/.kaggle/kaggle.json

In [ ]:
# commands to download data from kaggle
!kaggle competitions download -c 11785-hw1p2-f24

!unzip -qo /content/11785-hw1p2-f24.zip -d '/content'

# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [5]:
# Dataset class to load train and validation data

def add_noise(mfcc, noise_factor=0.005):
    noise = torch.randn(mfcc.size()) * noise_factor
    return mfcc + noise

def time_shift(mfcc, shift_limit=0.2):
    if mfcc.ndim == 1:
        return mfcc
    shift = int(torch.rand(1).item() * shift_limit * mfcc.size(1))
    if shift > 0:
        mfcc = torch.cat((mfcc[:, shift:], mfcc[:, :shift]), dim=1)
    elif shift < 0:
        mfcc = torch.cat((mfcc[:, shift:], mfcc[:, :shift]), dim=1)
    return mfcc

def time_mask(mfcc, mask_param=10):
    t = mfcc.size(0)
    t_mask = torch.randint(0, mask_param, (1,)).item()
    t_mask_start = torch.randint(0, max(1, t - t_mask), (1,)).item()
    mfcc[t_mask_start:t_mask_start + t_mask] = 0
    return mfcc

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes=PHONEMES, context=0, partition="train-clean-100", train=False):
        self.context = context
        self.phonemes = phonemes
        self.train = train
        self.mfcc_dir = os.path.join(root, partition, "mfcc")
        self.transcript_dir = os.path.join(root, partition, "transcript")

        mfcc_names = sorted(os.listdir(self.mfcc_dir))
        transcript_names = sorted(os.listdir(self.transcript_dir))
        assert len(mfcc_names) == len(transcript_names)
        self.mfccs, self.transcripts = [], []

        for i in range(len(mfcc_names)):
            mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_names[i]))
            mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)
            transcript = np.load(os.path.join(self.transcript_dir, transcript_names[i]))
            transcript = transcript[1:-1]
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        self.mfccs = np.concatenate(self.mfccs, axis=0)
        self.transcripts = np.concatenate(self.transcripts, axis=0)
        self.length = len(self.mfccs)
        self.mfccs = np.pad(self.mfccs, pad_width=((self.context, self.context), (0, 0)), mode='constant',
                            constant_values=0)

        phoneme_to_index = {phoneme: idx for idx, phoneme in enumerate(self.phonemes)}
        self.transcripts = np.array([phoneme_to_index[phoneme] for phoneme in self.transcripts])

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        frames = self.mfccs[ind: ind + 2 * self.context + 1]
        frames = frames.flatten()
        frames = torch.FloatTensor(frames)
        
        if self.train:
            # Apply data augmentations
            frames = add_noise(frames)
            frames = time_shift(frames)
            frames = time_mask(frames)
    
        phonemes = torch.tensor(self.transcripts[ind])

        return frames, phonemes

In [7]:
class AudioTestDataset(torch.utils.data.Dataset):
    def __init__(self, root, context=0, partition="test_clean"):
        self.context = context
        self.mfcc_dir = os.path.join(root, partition, "mfcc")
        mfcc_names = sorted(os.listdir(self.mfcc_dir))
        self.mfccs = []
        for i in range(len(mfcc_names)):
            mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_names[i]))
            mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)
            self.mfccs.append(mfcc)
        self.mfccs = np.concatenate(self.mfccs, axis=0)
        self.length = len(self.mfccs)
        self.mfccs = np.pad(self.mfccs, pad_width=((self.context, self.context), (0, 0)), mode='constant',
                            constant_values=0)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        frames = self.mfccs[ind: ind + 2 * self.context + 1]
        frames = frames.flatten()
        frames = torch.FloatTensor(frames)
        return frames

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments.

In [8]:
config = {
    'epochs': 20,
    'batch_size': 1024,
    'context': 20,
    'init_lr': 0.0015,
    'architecture': 'midium-cutoff',
    'dropout': 0.1
}

# Create Datasets

In [10]:
root_dir = "/content/11785-f24-hw1p2"
train_data = AudioDataset(root=root_dir, context=config['context'], partition="train-clean-100",train=True)
val_data = AudioDataset(root=root_dir, context=config['context'], partition="dev-clean")
test_data = AudioTestDataset(root=root_dir, context=config['context'],partition="test-clean")

In [11]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset=train_data,
    num_workers=4,
    batch_size=config['batch_size'],
    pin_memory=True,
    shuffle=True
)

val_loader = torch.utils.data.DataLoader(
    dataset=val_data,
    num_workers=2,
    batch_size=config['batch_size'],
    pin_memory=True,
    shuffle=False
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_data,
    num_workers=2,
    batch_size=config['batch_size'],
    pin_memory=True,
    shuffle=False
)

print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2 * config['context'] + 1) * 28)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  1024
Context        :  20
Input size     :  1148
Output symbols :  42


In [10]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([1024, 1148]) torch.Size([1024])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [12]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):
        super(Network, self).__init__()

    def __init__(self, input_size, output_size):
        super(Network, self).__init__()

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 3072),
            torch.nn.BatchNorm1d(3072),
            torch.nn.ReLU(),
            torch.nn.Dropout(config['dropout']),
            torch.nn.Linear(3072, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.ReLU(),
            torch.nn.Dropout(config['dropout']),
            torch.nn.Linear(2048, 1024),
            torch.nn.BatchNorm1d(1024),
            torch.nn.ReLU(),
            torch.nn.Dropout(config['dropout']),
            torch.nn.Linear(1024, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.ReLU(),
            torch.nn.Dropout(config['dropout']),
            torch.nn.Linear(512, 256),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU(),
            torch.nn.Dropout(config['dropout']),
            torch.nn.Linear(256, 128),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, output_size)
        )

    def forward(self, x):
        out = self.model(x)
        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler.

In [12]:
INPUT_SIZE = (2 * config['context'] + 1) * 28  # Why is this the case?
model = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 20 million parameters for HW1 (including ensembles)

----------------------------------------------------------------------------------------------------
Layer                   Kernel Shape         Output Shape         # Params (K)      # Mult-Adds (M)
0_Linear                [1148, 3072]         [1024, 3072]             3,529.73                 3.53
1_BatchNorm1d                 [3072]         [1024, 3072]                 6.14                 0.00
2_ReLU                             -         [1024, 3072]                    -                    -
3_Dropout                          -         [1024, 3072]                    -                    -
4_Linear                [3072, 2048]         [1024, 2048]             6,293.50                 6.29
5_BatchNorm1d                 [2048]         [1024, 2048]                 4.10                 0.00
6_ReLU                             -         [1024, 2048]                    -                    -
7_Dropout                          -         [1024, 2048]                    -                    -

In [13]:
criterion = torch.nn.CrossEntropyLoss()  # Defining Loss function.
# We use CE because the task is multi-class classification

optimizer = torch.optim.AdamW(model.parameters(), lr=config['init_lr'], weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=3, T_mult=1, eta_min=1e-6)

# Is your training time very high?
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [14]:
torch.cuda.empty_cache()
gc.collect()

4

In [15]:
def train(model, dataloader, optimizer, criterion, scheduler):
    model.train()
    tloss, tacc = 0, 0  # Monitoring loss and accuracy
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):
        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames = frames.to(device)
        phonemes = phonemes.to(device)

        ### Forward Propagation
        logits = model(frames)

        ### Loss Calculation
        loss = criterion(logits, phonemes)

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step()
        
        scheduler.step()

        tloss += loss.item()
        tacc += torch.sum(torch.argmax(logits, dim=1) == phonemes).item() / logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc * 100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    tloss /= len(train_loader)
    tacc /= len(train_loader)

    return tloss, tacc

In [16]:
def eval(model, dataloader):
    model.eval()  # set model in evaluation mode
    vloss, vacc = 0, 0  # Monitoring loss and accuracy
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):
        ### Move data to device (ideally GPU)
        frames = frames.to(device)
        phonemes = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits = model(frames)
            ### Loss Calculation
            loss = criterion(logits, phonemes)

        vloss += loss.item()
        vacc += torch.sum(torch.argmax(logits, dim=1) == phonemes).item() / logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc * 100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss /= len(val_loader)
    vacc /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb.

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [17]:
wandb.login(key="98642f33baa53793f08e5f32f1d09da8c7c6e80b")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [18]:
# Create your wandb run
run = wandb.init(
        reinit=True,
        project="hw1p2",  ### Project should be created in your wandb account
        config=config  ### Wandb Config for your run
    )

wandb: Currently logged in as: seitomoyi (seitomoyi-carnegie-mellon-university). Use `wandb login --relogin` to force relogin


In [19]:
### Save your model architecture as a string with str(model)
model_arch = str(model)

### Save it in a txt file
arch_file = open("model_arch.txt", "w")
file_write = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/kaggle/working/wandb/run-20240929_222832-sw92fhgv/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [20]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")
best_val_acc = 0

for epoch in range(config['epochs']):
    print("\nEpoch {}/{}".format(epoch + 1, config['epochs']))

    curr_lr = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, scheduler)
    val_train_loss, val_train_acc = train(model, val_loader, optimizer, criterion, scheduler)
    val_loss, val_acc = eval(model, val_loader)

    print(
        "\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc * 100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc * 100, val_loss))

    ### Log metrics at each epoch in your run
    # Optionally, you can log at each batch inside train/eval functions
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc * 100, 'train_loss': train_loss,
               'val_acc': val_acc * 100, 'valid_loss': val_loss, 'lr': curr_lr})

    if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f"/content/drive/MyDrive/HW1P2/best_model.pth")

    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best


  0%|          | 0/35246 [00:05<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7cc83ffd2680>
Exception ignored in: Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x7cc83ffd2680>
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
Traceback (most recent call last):
      File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    self._shutdown_workers()
self._shutdown_workers()
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers
    if w.is_alive():  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers

  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'    
AssertionError: can only test a child processif w.is_alive():

  File "/op

  0%|          | 0/1884 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7cc83ffd2680>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
Exception ignored in:     <function _MultiProcessingDataLoaderIter.__del__ at 0x7cc83ffd2680>
self._shutdown_workers()Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__

    self._shutdown_workers()  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers

      File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers
if w.is_alive():    
if w.is_alive():
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
      File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 

Epoch 1/20
	Train Acc 76.5884%	Train Loss 0.7233
	Val Acc 79.3407%	Val Loss 0.6193


...

  0%|          | 0/35246 [00:05<?, ?it/s]

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7cc83ffd2680>
<function _MultiProcessingDataLoaderIter.__del__ at 0x7cc83ffd2680>Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7cc83ffd2680>Exception ignored in: self._shutdown_workers()Traceback (most recent call last):

  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
<function _MultiProcessingDataLoaderIter.__del__ at 0x7cc83ffd2680>Traceback (most recent call last):

      File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers
  File "/opt/conda/lib/python3.10/site-packag

  0%|          | 0/1884 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7cc83ffd2680>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers
    if w.is_alive():
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    AssertionErrorassert self._parent_pid == os.getpid(), 'can only test a child process'
: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7cc83ffd2680>can only test a child process

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers
    if w.is_alive():
  File "/op

Epoch 20/20
	Train Acc 88.0775%	Train Loss 0.3371
	Val Acc 87.7102%	Val Loss 0.3460


# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [1]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.no_grad(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)

            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            predicted_phonemes = torch.argmax(logits, dim= 1) #NotImplemented

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval
            # TODO
            test_predictions.extend(np.array(PHONEMES)[predicted_phonemes.cpu().numpy()])

    return test_predictions

In [13]:
INPUT_SIZE = (2 * config['context'] + 1) * 28  # Why is this the case?
model = Network(INPUT_SIZE, 42).to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/HW1P2/best_model.pth"))
model.eval()
predictions = test(model, test_loader)

/tmp/ipykernel_30/3280655040.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/working/best_model_87.71016967084395_0.0015_1024_0

  0%|          | 0/1889 [00:00<?, ?it/s]

In [14]:
### Create CSV file with predictions
with open("/content/drive/MyDrive/HW1P2/submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [ ]:
### Finish your wandb run
run.finish()

In [ ]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
# !kaggle competitions submit -c 11785-hw1p2-f24 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle